# Cross Validation: ARIMA

Simple ARIMA (p,d,q)(P,D,Q,m) model.  Order and seasonal order chosen using `pmdarima.auto_arima`

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#stats models imports for tsa
from statsmodels.tsa.arima.model import ARIMA

#auto_arima
from pmdarima import auto_arima


#forecast error metrics
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

import warnings
warnings.filterwarnings('ignore')

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [2]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'arima'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [3]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [4]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [5]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [6]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [7]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#train split into train and validation
train, val = ts_train_test_split(train, split_date=VAL_SPLIT_DATE)

In [8]:
train.shape

(1279, 9)

In [9]:
val.shape

(549, 9)

# Automatic model selection

Uses Auto ARIMA function to select model using the Hyndman-

In [10]:
##only looking at Trust level.
#auto_results = auto_arima(train[REGION], seasonal=True, m=7, n_job=-1, 
#                          suppress_warnings=True) 
#auto_results.summary()

## Time Series Cross Validation

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

## ARIMA Wrapper Class
`ARIMAWrapper` is a facade for `ARIMA`.  Provides additional flexibility for ARIMA objects to fit in with common TSCV across methods.

In [11]:
class ARIMAWrapper:
    '''
    Facade for statsmodels.statespace
    '''
    def __init__(self, order, seasonal_order, training_index, holidays=None):
        self._order = order
        self._seasonal_order = seasonal_order
        self._training_index = training_index
        self._holidays = holidays

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues
    
    def _encode_holidays(self, holidays, idx):
        dummy = idx.isin(holidays).astype(int)
        dummy = pd.DataFrame(dummy)
        dummy.columns = ['holiday']
        dummy.index = idx
        return dummy

    def fit(self, y_train):
        
        #extend training index
        if len(y_train) > len(self._training_index):

            self._training_index = pd.date_range(start=self._training_index[0], 
                                                 periods=len(y_train),
                                                 freq=self._training_index.freq)
            
        holiday_train = None
        if not self._holidays is None:
            holiday_train = self._encode_holidays(self._holidays, 
                                                  self._training_index)
    
        
        self._model = ARIMA(endog=y_train,
                            exog=holiday_train,
                            order=self._order, 
                            seasonal_order=self._seasonal_order)#,
                            #enforce_stationarity=False)
        
        self._fitted = self._model.fit()
        self._t = len(train)
        
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        forecast h steps ahead.
        
        Params:
        ------
        h: int
            h-step forecast
        
        return_conf_int: bool, optional (default=False)
            return 1 - alpha PI
        
        alpha: float, optional (default=0.2)
            return 1 - alpha PI
                       
        Returns:
        -------
        np.array
            If return_conf_int = False returns preds only
            
        np.array, np.array
            If return_conf_int = True returns tuple of preds, pred_ints
        '''
        
        #+1 to date range then trim off the first value

        f_idx = pd.date_range(start=self._training_index[-1], 
                              periods=horizon+1,
                              freq=self._training_index.freq)[1:]
        
        #encode holidays if included.
        exog_holiday = None
        if not self._holidays is None:
            exog_holiday = self._encode_holidays(self._holidays, f_idx)
        
    
        forecast = self._fitted.get_forecast(horizon, exog=exog_holiday)
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi
            
        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)  

In [12]:
def time_series_cv(model, train, val, horizons, alpha=0.2, step=1):
    '''
    Time series cross validation across multiple horizons for a single model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model - forecasting model

    error_func - function to measure forecast error

    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days

    step -- step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array - vector of forecast errors from the CVs.
    '''
    cv_preds = [] #mean forecast
    cv_actuals = [] # actuals 
    cv_pis = [] #prediction intervals
    split = 0

    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
                
        train_cv = np.concatenate([train, val[:i]], axis=0)
        model.fit(train_cv)
        
        #predict the maximum horizon 
        preds, pis = model.predict(horizon=len(val[i:i+max(horizons)]), 
                                   return_conf_int=True,
                                   alpha=alpha)
        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h])    
            cv_h_pis.append(pis[:h])
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [13]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [14]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

# Run Cross Validation

This is run twices once each for 80 and 95% prediction intervals.

In [15]:
model = ARIMAWrapper(order=(1,1,3), seasonal_order=(1,0,1,7), 
                     training_index=train.index )
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.2, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [16]:
cv_preds, cv_test, cv_intervals = results

In [17]:
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,3.035132,3.392277,3.583495,3.729142,3.865357,3.978708,4.065642,4.137531,4.220929,4.273174,4.321929,4.395273,4.940061
std,1.262805,1.281414,1.188705,1.138401,1.183108,1.191705,1.171287,1.118916,1.088968,1.022889,0.980988,1.002567,1.450380
min,1.577649,1.683922,2.208552,2.545710,2.569328,2.623963,2.547736,2.667818,2.850693,3.028543,2.973107,3.002794,3.675961
25%,2.093875,2.645470,2.782894,2.800936,2.946880,3.126364,3.286685,3.286695,3.450556,3.630111,3.728494,3.681126,3.926416
50%,2.826170,2.989510,3.160257,3.425163,3.399747,3.542341,3.582434,3.905213,3.867323,3.977032,4.007547,4.075594,4.939773
75%,3.588407,3.731665,3.963487,4.159276,4.617119,4.528372,4.461585,4.398957,4.510204,4.868421,4.986627,5.261835,5.308781
max,6.726198,7.328785,6.641379,6.284334,6.660583,6.948891,6.637317,6.980925,6.634661,6.213411,5.920411,6.277706,10.961270


In [18]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima_smape.csv


In [19]:
#CV point predictions rmse
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,81.328904,92.775766,99.883690,104.923147,109.545176,113.361307,116.520853,119.278348,122.305484,124.653390,126.789933,129.456951,142.349451
std,38.906192,41.974036,41.129839,40.299836,40.687146,40.414737,39.749977,38.516219,37.260434,35.419301,34.033853,33.596446,32.849969
min,37.749829,43.520465,60.417265,69.166256,73.204175,74.015983,72.642331,74.519751,78.691607,81.503218,80.722929,82.153078,114.233418
25%,57.800416,71.033397,75.189682,81.028438,80.303346,85.010126,85.740147,88.909605,93.529514,99.041547,101.323494,103.663933,120.374767
50%,74.097905,81.690905,88.022121,89.202388,90.776345,93.853076,97.465912,103.750943,109.981670,116.709021,122.733264,127.627702,139.158825
75%,89.501130,92.777513,102.352072,106.782087,118.676985,143.226686,149.844926,149.473017,147.418253,149.612130,156.014156,158.305824,151.045195
max,225.863437,241.936372,212.734803,209.510880,209.277748,209.924755,201.874777,206.106965,198.716598,189.972576,183.067628,186.420896,280.863006


In [20]:
#output rmse
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima_rmse.csv


In [21]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train[REGION])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.822299,0.921157,0.974330,1.014737,1.052366,1.083733,1.107944,1.127696,1.150705,1.165272,1.179067,1.199488,1.341217
std,0.387519,0.398337,0.372758,0.358631,0.367517,0.366709,0.358215,0.341111,0.329969,0.308926,0.293755,0.295900,0.414752
min,0.412711,0.444991,0.593176,0.646963,0.663596,0.675018,0.653409,0.683681,0.734620,0.783252,0.768834,0.781345,0.999604
25%,0.549874,0.698068,0.712893,0.740457,0.784455,0.822715,0.849610,0.865793,0.912621,0.957298,0.992833,1.017318,1.066549
50%,0.765355,0.810057,0.822050,0.901893,0.915998,0.910270,0.953313,1.051872,1.108253,1.101217,1.111078,1.145994,1.328691
75%,0.941207,0.998817,1.082702,1.133929,1.269883,1.297610,1.283175,1.231585,1.253561,1.338862,1.375398,1.435912,1.425041
max,2.074037,2.204953,1.922648,1.840182,1.921905,1.980679,1.881645,1.960930,1.857206,1.745497,1.676137,1.779063,3.105663


In [22]:
#output rmse
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima_mase.csv


In [23]:
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.814815,0.780423,0.770723,0.761905,0.747090,0.738095,0.733938,0.727513,0.720165,0.717460,0.715729,0.711640,0.775342
std,0.197372,0.173755,0.146496,0.135454,0.146053,0.149859,0.147435,0.137484,0.135057,0.128164,0.123223,0.124172,0.113929
min,0.285714,0.285714,0.333333,0.428571,0.371429,0.357143,0.387755,0.392857,0.412698,0.457143,0.506494,0.476190,0.284932
25%,0.714286,0.750000,0.690476,0.625000,0.671429,0.702381,0.693878,0.687500,0.650794,0.628571,0.597403,0.595238,0.743836
50%,0.857143,0.857143,0.809524,0.821429,0.800000,0.785714,0.755102,0.767857,0.777778,0.757143,0.740260,0.750000,0.813699
75%,1.000000,0.857143,0.857143,0.857143,0.857143,0.857143,0.857143,0.830357,0.801587,0.807143,0.811688,0.821429,0.846575
max,1.000000,1.000000,0.952381,0.928571,0.914286,0.904762,0.897959,0.892857,0.904762,0.900000,0.896104,0.892857,0.871233


In [24]:
#output 80% PI coverage
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima_coverage_80.csv


In [25]:
#95% PIs
model = ARIMAWrapper(order=(1,1,3), seasonal_order=(1,0,1,7), 
                     training_index=train.index )
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.05, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [26]:
cv_preds_95, cv_test_95, cv_intervals_95 = results

In [27]:
cv_coverage_95 = prediction_int_coverage_cv(cv_test_95, cv_intervals_95)
df_95 = pd.DataFrame(cv_coverage_95)
df_95.columns = horizons
df_95.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.962963,0.947090,0.945326,0.943122,0.938624,0.935626,0.931973,0.931217,0.927102,0.924339,0.921597,0.916667,0.936986
std,0.093699,0.114821,0.103744,0.094106,0.087779,0.082430,0.080247,0.077791,0.071294,0.063742,0.058689,0.056033,0.036259
min,0.571429,0.500000,0.666667,0.714286,0.742857,0.738095,0.775510,0.750000,0.777778,0.800000,0.818182,0.821429,0.791781
25%,1.000000,0.928571,0.952381,0.964286,0.957143,0.869048,0.846939,0.857143,0.873016,0.878571,0.863636,0.869048,0.923288
50%,1.000000,1.000000,1.000000,1.000000,0.971429,0.976190,0.979592,0.982143,0.968254,0.942857,0.935065,0.916667,0.945205
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.982143,0.984127,0.985714,0.974026,0.970238,0.958904
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.988095,0.972603


In [29]:
#output 95% PI coverage
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df_95.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima_coverage_95.csv


# End